In [ ]:
# Required Libraries

import cv2                                  # used for video capture and object detection
import imutils                              # used for image processing as resizing
import datetime                             # used to compute time
import numpy as np
from centroidtracker import CentroidTracker # used for object tracking

In [ ]:
# MobilenetSSD model for object detection
# that computes the bounding box and category of an object

protopath = "MobileNetSSD_deploy.prototxt"
modelpath = "MobileNetSSD_deploy.caffemodel"
detector = cv2.dnn.readNetFromCaffe(prototxt=protopath, caffeModel=modelpath)

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

In [ ]:
# function that selects the best bounding box out of a set of overlapping boxes

def non_max_suppression_fast(boxes, overlapThresh):
    try:
        if len(boxes) == 0:
            return []

        if boxes.dtype.kind == "i":
            boxes = boxes.astype("float")

        pick = []

        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)

        while len(idxs) > 0:
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)

            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])

            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)

            overlap = (w * h) / area[idxs[:last]]

            idxs = np.delete(idxs, np.concatenate(([last],
                                                   np.where(overlap > overlapThresh)[0])))

        return boxes[pick].astype("int")
    except Exception as e:
        print("Exception occurred in non_max_suppression : {}".format(e))

In [ ]:
#Task 2

def Persons_count():
    cap = cv2.VideoCapture('vtest.avi') 

    total_frames = 0

    
    object_id_list = []
    dtime = dict()
    dwell_time = dict()
    persons_count = []
    
    while True:
        ret, frame = cap.read() #read from video file
        
        frame = imutils.resize(frame, width=600) #resizing video frame
        
        total_frames = total_frames + 1

        (H, W) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5) #returns a blob, pre-processing Image

        detector.setInput(blob) #passing blob through detector
        person_detections = detector.forward() 
        
        
        # considering detections with confidence greater than 40
        rects = []
        for i in np.arange(0, person_detections.shape[2]):
            confidence = person_detections[0, 0, i, 2]
            if confidence > 0.4:
                idx = int(person_detections[0, 0, i, 1]) #detections index 

                if CLASSES[idx] != "person":
                    continue

                person_box = person_detections[0, 0, i, 3:7] * np.array([W, H, W, H]) 
                (startX, startY, endX, endY) = person_box.astype("int") #bounding box co-ordinates
                rects.append(person_box)
        
              
        boundingboxes = np.array(rects)
        boundingboxes = boundingboxes.astype(int)
        
        # reducing noise within boundary boxes
        rects = non_max_suppression_fast(boundingboxes, 0.3)
        
        
        tracker = CentroidTracker(maxDisappeared=50, maxDistance=50)
        
        # object Id and bounding box
        objects = tracker.update(rects)
        for (objectId, bbox) in objects.items():
            x1, y1, x2, y2 = bbox
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)
            
            
            if objectId not in object_id_list:
                    object_id_list.append(objectId)
                    dtime[objectId] = datetime.datetime.now()
                    dwell_time[objectId] = 0
            else:
                present_time = datetime.datetime.now()
                old_time = dtime[objectId]
                time_diff = present_time - old_time
                dtime[objectId] = datetime.datetime.now()
                dwell_time[objectId] += time_diff.total_seconds()
            
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)


        # total count of persons in video
        persons_count = len(objects)
        persons_count_text = "Persons Count: {}".format(persons_count)
        cv2.putText(frame, persons_count_text, (5, 60), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 255, 0), 2)
 

        #alert when more than three persons stand together a time more than 20 seconds.
        if persons_count > 3 and dwell_time[objectId] > 5:
            temp = "ALERT!!!"
            cv2.putText(frame,temp,(300,50),cv2.FONT_HERSHEY_DUPLEX,1.5,(0,0,255),3)

        cv2.imshow("Persons Count Application", frame)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
                    
                    
    cv2.imshow("Persons Count", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Persons_count()

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to tar

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to target thread (0x55b52aa621a0)

QObject::moveToThread: Current thread (0x55b52aa621a0) is not the object's thread (0x55b52caa3a50).
Cannot move to tar